In [4]:
# NOTEBOOK METEOSTAT - PHIÊN BẢN MINH BẠCH
# CELL 1: Khám phá các trạm thời tiết

import pandas as pd
from meteostat import Stations

# Tọa độ trung tâm Hà Nội
HANOI_LAT = 21.0285
HANOI_LON = 105.8542

# 1. Khởi tạo đối tượng Stations
stations = Stations()

# 2. Tìm kiếm các trạm trong bán kính 100km quanh Hà Nội
# Điều này sẽ cho chúng ta thấy tất cả các lựa chọn có thể có
nearby_stations = stations.nearby(HANOI_LAT, HANOI_LON, 100000) # 100km = 100,000m

# 3. Lấy thông tin chi tiết của các trạm tìm thấy
df_stations_info = nearby_stations.fetch()

print("---> DANH SÁCH CÁC TRẠM THỜI TIẾT METEOSTAT TÌM THẤY GẦN HÀ NỘI <---")

if df_stations_info.empty:
    print("Không tìm thấy trạm nào.")
else:
    # In ra các cột quan trọng
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        print(df_stations_info[['name', 'country', 'latitude', 'longitude', 'elevation']])

---> DANH SÁCH CÁC TRẠM THỜI TIẾT METEOSTAT TÌM THẤY GẦN HÀ NỘI <---
                                      name country  latitude  longitude  elevation
id                                                                                
VVGL0                      Hanoi / Xóm Pho      VN   21.0333   105.8500       12.0
48820                               Ha Noi      VN   21.0167   105.8000        6.0
48825  Haiphong / Cat Bi / Catbi / Dong Ya      VN   20.9667   105.7667        6.0
48826                             Phu Lien      VN   20.8000   106.6333      119.0


In [5]:
# NOTEBOOK METEOSTAT
# CELL 2 (Tinh chỉnh cuối cùng)

from meteostat import Stations
import pandas as pd
import os
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re

# --- THÔNG SỐ BẠN QUYẾT ĐỊNH ---
# Dựa trên kết quả tải dữ liệu, chúng ta loại bỏ VVGL0 vì nó không có dữ liệu hourly
VALID_WEATHER_STATION_IDS = ['48820', '48825']
# --------------------------------

MAX_DISTANCE_KM = 30 
metadata_filename = "../stations_metadata.csv"

if not os.path.exists(metadata_filename):
    print("Lỗi: File metadata không tồn tại.")
else:
    df_metadata = pd.read_csv(metadata_filename)
    
    # ... (Toàn bộ phần code còn lại của Cell 2 giữ nguyên) ...
    # ... (Tìm tên thật, Tạo bản đồ liên kết) ...
    
    # --- PHẦN TÌM TÊN THẬT ---
    print("--- Bước 1: Tìm tên thật cho các trạm thời tiết tham chiếu ---")
    stations = Stations()
    geolocator = Nominatim(user_agent="hanoi_aq_project_v3")
    geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
    weather_station_name_map = {}
    all_stations_df = stations.fetch()
    
    for station_id in VALID_WEATHER_STATION_IDS:
        station_info = all_stations_df[all_stations_df.index == station_id]
        if not station_info.empty:
            lat = station_info['latitude'].values[0]
            lon = station_info['longitude'].values[0]
            try:
                location = geocode((lat, lon), language='en', exactly_one=True)
                address = location.raw.get('address', {})
                city_district = address.get('city_district', address.get('suburb', address.get('city', 'Unknown')))
                country_code = address.get('country_code', 'un').upper()
                clean_name = f"{city_district.replace(' ', '_')}_{country_code}".lower()
                clean_name = re.sub(r'[\W_]+', '_', clean_name).strip('_')
                weather_station_name_map[station_id] = clean_name
                print(f"Trạm ID {station_id} -> Tên thật: {city_district} -> Tên file: {clean_name}")
            except Exception as e:
                weather_station_name_map[station_id] = station_id
        else:
            weather_station_name_map[station_id] = station_id
            
    # --- PHẦN TẠO BẢN ĐỒ ---
    print("\n--- Bước 2: Tạo bản đồ liên kết trạm ô nhiễm và trạm thời tiết ---")
    discovery_results = []
    for index, aq_station in df_metadata.iterrows():
        nearby_stations_df = stations.nearby(aq_station['lat'], aq_station['lon']).fetch()
        valid_nearby_stations = nearby_stations_df[nearby_stations_df.index.isin(VALID_WEATHER_STATION_IDS)]
        closest_station_info = valid_nearby_stations.head(1)
        
        if not closest_station_info.empty:
            distance_km = closest_station_info['distance'].values[0] / 1000
            if distance_km <= MAX_DISTANCE_KM:
                discovery_results.append({
                    "aq_station_id": aq_station['location_id'],
                    "assigned_weather_station_id": closest_station_info.index[0]
                })

    df_mapping = pd.DataFrame(discovery_results)
    mapping_filename = "station_mapping_meteostat.csv"
    df_mapping.to_csv(mapping_filename, index=False, encoding='utf-8-sig')
    print(f"Đã tạo và lưu bản đồ liên kết (chỉ với các trạm có dữ liệu) vào file: '{mapping_filename}'")
    print(df_mapping.head())

--- Bước 1: Tìm tên thật cho các trạm thời tiết tham chiếu ---
Trạm ID 48820 -> Tên thật: Yen Hoa Ward -> Tên file: yen_hoa_ward_vn
Trạm ID 48825 -> Tên thật: Phường Hà Đông -> Tên file: phường_hà_đông_vn

--- Bước 2: Tạo bản đồ liên kết trạm ô nhiễm và trạm thời tiết ---
Đã tạo và lưu bản đồ liên kết (chỉ với các trạm có dữ liệu) vào file: 'station_mapping_meteostat.csv'
   aq_station_id assigned_weather_station_id
0           2539                       48820
1           7441                       48820
2        1285357                       48820
3        2161290                       48825
4        2161291                       48820


In [6]:
# NOTEBOOK METEOSTAT
# CELL 3 (Cập nhật): Thu thập dữ liệu và lưu với tên file mới

from datetime import datetime
from meteostat import Hourly
import time

if 'VALID_WEATHER_STATION_IDS' not in locals() or 'weather_station_name_map' not in locals():
    print("Vui lòng chạy Cell 2 để xác định các trạm và tên thật của chúng.")
else:
    ms_output_dir = "hanoi_weather_meteostat_ref_raw"
    if not os.path.exists(ms_output_dir): os.makedirs(ms_output_dir)

    START_DATE = datetime(2022, 1, 1)
    END_DATE = datetime.now()

    print(f"\nBắt đầu quá trình tải dữ liệu cho {len(VALID_WEATHER_STATION_IDS)} trạm tham chiếu...")
    
    for station_id in VALID_WEATHER_STATION_IDS:
        # Lấy tên thật đã được chuẩn hóa từ sổ tay
        clean_name = weather_station_name_map.get(station_id, station_id)
        output_filename = os.path.join(ms_output_dir, f"weather_{station_id}_{clean_name}.csv")

        if os.path.exists(output_filename):
            print(f"\n--- File cho trạm {station_id} ({clean_name}) đã tồn tại. Bỏ qua. ---")
            continue
            
        print(f"\n--- Đang xử lý trạm thời tiết ID: {station_id} ({clean_name}) ---")

        try:
            data = Hourly(station_id, START_DATE, END_DATE)
            df_weather = data.fetch()

            if not df_weather.empty:
                # ... (phần code đổi tên cột giữ nguyên như cũ) ...
                df_weather = df_weather.rename(columns={
                    'temp': 'temp_ref', 'rhum': 'rhum_ref', 'prcp': 'prcp_ref',
                    'pres': 'pres_ref', 'wspd': 'wspd_ref', 'wdir': 'wdir_ref'
                })
                df_weather = df_weather.loc[:, df_weather.columns.isin(['temp_ref', 'rhum_ref', 'prcp_ref', 'pres_ref', 'wspd_ref', 'wdir_ref'])]

                df_weather.to_csv(output_filename, encoding='utf-8-sig')
                print(f"    -> Đã lưu thành công {len(df_weather)} giờ dữ liệu vào file: {output_filename}")
            else:
                print("    -> Không tìm thấy dữ liệu.")
            
            time.sleep(1)

        except Exception as e:
            print(f"    -> Đã xảy ra lỗi: {e}")

    print("\n\n---> HOÀN TẤT QUÁ TRÌNH TẢI DỮ LIỆU THAM CHIẾU <---")


Bắt đầu quá trình tải dữ liệu cho 2 trạm tham chiếu...

--- Đang xử lý trạm thời tiết ID: 48820 (yen_hoa_ward_vn) ---
    -> Đã lưu thành công 33034 giờ dữ liệu vào file: hanoi_weather_meteostat_ref_raw\weather_48820_yen_hoa_ward_vn.csv

--- Đang xử lý trạm thời tiết ID: 48825 (phường_hà_đông_vn) ---
    -> Đã lưu thành công 33027 giờ dữ liệu vào file: hanoi_weather_meteostat_ref_raw\weather_48825_phường_hà_đông_vn.csv


---> HOÀN TẤT QUÁ TRÌNH TẢI DỮ LIỆU THAM CHIẾU <---
